# Workflow to Validate NISAR L2 Transient Displacement Requirements

**Original code authored by:** NISAR Science Team Members and Affiliates  

*May 13, 2022*

*NISAR Solid Earth Team*

<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. And all subsequent sections NEED to be run in order.
</div>

<hr/>

## Define CalVal Site 

In [ ]:
# Choose a site and track direction
site='test' 

# Choose the requirement to validate
# Options: 'Secular' 'Coseismic' 'Transient'
requirement = 'Transient'

# What dataset are you processing?
# Options: 
#'ARIA_S1_new' (***reorganized! for Sentinel-1 testing with aria-tools)
#'ARIA_S1' (for Sentinel-1 testing with aria-tools)
dataset = 'ARIA_S1_new'
aria_gunw_version = '3_0_1'

# The date and version of this Cal/Val run
today = '20240429'
version = '1'

# Define your directory structure - you won't need to change this line
start_directory = '/scratch/nisar-st-calval-solidearth' 

# The file where you keep your customized list of sites.
custom_sites = '/home/jovyan/my_sites.txt'

# Please enter a name or username that will determine where your outputs are stored
import os
if os.path.exists('/home/jovyan/me.txt'): # if OpenTopo API key already installed
    with open('/home/jovyan/me.txt') as m:
        you = m.readline().strip()
    print('You are', you)
    print('Using this as the name of the directory where your outputs will be stored.')
    print('Directory structure: start_directory / dataset/ requirement / site / you / today / version ')

else:
    print('We need a name or username (determines where your outputs will be stored)')
    print('Directory structure: start_directory / dataset/ requirement / site / you / today / version ')

    you = input('Please type your name:')
    with open ('/home/jovyan/me.txt', 'w') as m: 
        m.write(you)
    

## Table of Contents:
<a id='secular_TOC'></a>

<hr/>

[**Prep A. Environment Setup**](#transient_prep_a)

[**Prep B. Data Staging**](#transient_prep_b)

[**1. Generate Interferogram Stack**](#transient_gen_ifg)
- [1.1.  Crop Interferograms](#transient_crop_ifg)

[**2. Optional Corrections**](#transient_opt_correction)
- [2.1. Solid Earth Tides Correction](#transient_solid_earth)
- [2.2. Tropospheric Delay Correction](#transient_tropo_corr)
- [2.3. Topographic Residual Correction ](#transient_tropo_res_corr)

[**3. Make GNSS LOS Measurements**](#transient_gnss_los)
- [3.1. Find Collocated GNSS Stations](#transient_co_gnss)  
- [3.2. Make GNSS LOS Measurements](#transient_gnss_los2) 
- [3.3. Make GNSS and InSAR Relative Displacements](#transient_gnss_insar)

[**4. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison**](#transient_validation1)
- [4.1. Pair up GNSS stations and make measurement residuals](#transient_pair1)
- [4.2. Validate the requirement based on binned measurement residuals](#transient_bin1)
- [4.3. Result visulazation](#transient_result1)
- [4.3. Conclusion](#transient_conclusion1)

[**5. NISAR Validation Approach 2: Noise Level Validation**](#transient_validation2)
- [5.1. Randomly sample pixels and pair them up](#transient_pair2)
- [5.2. Validate the requirement based on binned measurement residuals](#transient_bin2)
- [5.3. Result visulazation](#transient_result2)
- [5.3. Conclusion](#transient_conclusion2)

[**6. Appendix: GNSS Position Plots**](#transient_appendix)

<hr/>

<a id='transient_prep_a'></a>
## Prep A. Environment Setup
Setup your environment for processing data

In [ ]:
# load packages
import copy
import glob
import math
import os
import random
import subprocess
from datetime import datetime as dt
from pathlib import Path
import json

import numpy as np
import pandas as pd
import pyproj
from matplotlib import pyplot as plt
from mintpy import smallbaselineApp
from mintpy.objects import gnss
from mintpy.utils import readfile, utils as ut

from solid_utils.sampling import load_geo, samp_pair

In [ ]:
################# Set Directories ##########################################

# Site directory
site_dir = os.path.join(start_directory, dataset, site)

work_dir = os.path.join(site_dir,requirement,you,today,'v'+version)
print("Work directory:", work_dir)

gunw_dir = os.path.join(site_dir,'products')
print("   GUNW    dir:", gunw_dir) 

mintpy_dir = os.path.join(work_dir,'MintPy')
print("   MintPy  dir:", mintpy_dir)

### Change to MintPy workdir
if not os.path.exists(mintpy_dir):
    print()
    print('ERROR: Stop! Your MintPy processing directory does not exist for this requirement, site, version, or date of your ATBD run.')
    print('You may need to run the prep notebook first!')
    print()
else:
    os.chdir(mintpy_dir)

vel_file = os.path.join(mintpy_dir, 'velocity.h5')
msk_file = os.path.join(mintpy_dir, 'maskConnComp.h5')  # maskTempCoh.h5 maskConnComp.h5

with open(custom_sites,'r') as fid:
    sitedata = json.load(fid)

sitedata['sites'][site]

In [ ]:
ifgs_file = os.path.join(mintpy_dir,'inputs/ifgramStack.h5')
geom_file = os.path.join(mintpy_dir,'inputs/geometryGeo.h5')

**NOTE:** If the interferogram has a resolution lower than 100 m, we need multi-look the interferogram phase values before calculating the empirical semivarigram.

Load the date of interferograms into Python:

In [ ]:
ifgs_date = readfile.read(ifgs_file,datasetName='date')[0]

In [ ]:
_ifgs_date = np.empty_like(ifgs_date,dtype=dt)
for i in range(ifgs_date.shape[0]):
    start_date = ifgs_date[i,0].decode()
    end_date = ifgs_date[i,1].decode()
    start_date = dt.strptime(start_date, "%Y%m%d")
    end_date = dt.strptime(end_date, "%Y%m%d")
    _ifgs_date[i] = [start_date,end_date]
    
ifgs_date = _ifgs_date
del _ifgs_date

Remove interferograms with time interval other than 12 days:

In [ ]:
del_row_index = []
for i in range(ifgs_date.shape[0]):
    time_interval = (ifgs_date[i,1]-ifgs_date[i,0]).days
    if time_interval != 12:
        del_row_index.append(i)
while i<ifgs_date.shape[0]-1:
    if ifgs_date[i,1]==ifgs_date[i+1,0]:
        del_row_index.append(i+1)
        i = i+2
    else:
        i = i+1

In [ ]:
ifgs_date = np.delete(ifgs_date,del_row_index,0)

Identify independent interferograms (i.e., selected inteferograms do NOT share common dates):

In [ ]:
del_row_index = []
i = 0
while i<ifgs_date.shape[0]-1:
    if ifgs_date[i,1]==ifgs_date[i+1,0]:
        del_row_index.append(i+1)
        i = i+2
    else:
        i = i+1

In [ ]:
ifgs_date = np.delete(ifgs_date,del_row_index,0)

Then the phase and coherence of selected interferograms, geometrical datasets, and attribution of them are loaded into numpy array:

In [ ]:
unwrapPhaseName = ['unwrapPhase-'+i[0].strftime('%Y%m%d')+'_'+i[1].strftime('%Y%m%d') for i in ifgs_date]
coherenceName = ['coherence-'+i[0].strftime('%Y%m%d')+'_'+i[1].strftime('%Y%m%d') for i in ifgs_date]

In [ ]:
ifgs_unw,atr = readfile.read(ifgs_file,datasetName=unwrapPhaseName)
insar_displacement = -ifgs_unw*float(atr['WAVELENGTH'])/(4*np.pi)*1000 # unit in mm

insar_coherence = readfile.read(ifgs_file,datasetName=coherenceName)[0]
del ifgs_unw

Change default missing phase values in interferograms from 0.0 to `np.nan`.

In [ ]:
insar_displacement[insar_displacement==0.0] = np.nan

<a id='transient_opt_correction'></a>
# Optional interferograms correction

Phase distortions related to solid earth and ocean tidal effects as well as those due to temporal variations in the vertical stratification of the atmosphere can be mitigated using the approaches described below. At this point, it is expected that these corrections will not be needed to validate the mission requirements, but they may be used to produce the highest quality data products. Typically, these are applied to the estimated time series product rather than to the individual interferograms, since they are a function of the time of each radar acquisition.

<a id='transient_solid_earth'></a>
## Solid Earth Tides Correction
[MintPy provides functionality for this correction.]

<a id='transient_tropo_corr'></a>
## Tropospheric Delay Correction
Optional atmospheric correction utilizes the PyAPS (Jolivet et al., 2011, Jolivet and Agram, 2012) module within GIAnT (or eventually a merged replacement for GIAnT and MintPy). PyAPS is well documented, maintained and can be freely downloaded. PyAPS is included in GIAnT distribution). PyAPS currently includes support for ECMWF’s ERA-Interim, NOAA’s NARR and NASA’s MERRA weather models. A final selection of atmospheric models to be used for operational NISAR processing will be done during Phase C.

[T]ropospheric delay maps are produced from atmospheric data provided by Global Atmospheric Models. This method aims to correct differential atmospheric delay correlated with the topography in interferometric phase measurements. Global Atmospheric Models (hereafter GAMs)... provide estimates of the air temperature, the atmospheric pressure and the humidity as a function of elevation on a coarse resolution latitude/longitude grid. In PyAPS, we use this 3D distribution of atmospheric variables to determine the atmospheric phase delay on each pixel of each interferogram.

The absolute atmospheric delay is computed at each SAR acquisition date. For a pixel a_i at an elevation z at acquisition date i, the four surrounding grid points are selected and the delays for their respective elevations are computed. The resulting delay at the pixel a_i is then the bilinear interpolation between the delays at the four grid points. Finally, we combine the absolute delay maps of the InSAR partner images to produce the differential delay maps used to correct the interferograms.

[MintPy provides functionality for this correction.]

<a id='transient_tropo_res_corr'></a>
## Topographic Residual Correction 
[MintPy provides functionality for this correction.]

**NOTE:** Phase deramping is not appplied here.

**NOTE:** If the solid earth tides correction for interferogram is applied, it should also be applied for GNSS observation.

Preliminary summary: we have load all data we need for processing:
- `atr`: metadata, including incident angle, longitude and latitude step width, etc;
- `insar_displacement`: LOS measurement from InSAR;
- `insar_coherence`: coherence value of the interferograms:
- `ifgs_date`: list of date pairs of two SAR images that form a interferogram.

<a id='transient_gnss_los'></a>
# Make GNSS LOS Measurements

<a id='transient_co_gnss'></a>
## Find Collocated GNSS Stations
The project will have access to L2 position data for continuous GNSS stations in third-party networks such NSF’s Plate Boundary Observatory, the HVO network for Hawaii, GEONET-Japan, and GEONET-New Zealand, located in target regions for NISAR solid earth calval. Station data will be post-processed by one or more analysis centers, will be freely available, and will have latencies of several days to weeks, as is the case with positions currently produced by the NSF’s GAGE Facility and separately by the University of Nevada Reno. Networks will contain one or more areas of high-density station coverage (2~20 km nominal station spacing over 100 x 100 km or more) to support validation of L2 NISAR requirements at a wide range of length scales.

Get space and time range for searching GNSS station:

In [ ]:
length, width = int(atr['LENGTH']), int(atr['WIDTH'])
lat_step = float(atr['Y_STEP'])
lon_step = float(atr['X_STEP'])
N = float(atr['Y_FIRST'])
W = float(atr['X_FIRST'])
S = N+lat_step*(length-1)
E = W+lon_step*(width-1)

In [ ]:
start_date_gnss = ifgs_date[0,0]
end_date_gnss = ifgs_date[-1,-1]

Search for collocated GNSS stations:

In [ ]:
os.chdir(mintpy_dir)
site_names, site_lats, site_lons = gnss.search_gnss(SNWE=(S,N,W,E),
                                                  start_date=start_date_gnss.strftime('%Y%m%d'),
                                                  end_date=end_date_gnss.strftime('%Y%m%d'))
os.chdir(work_dir)
site_names = [str(stn) for stn in site_names]
print("Initial list of {} stations used in analysis:".format(len(site_names)))
print(site_names)

<a id='transient_gnss_los2'></a>
## Make GNSS LOS Measurements

In this step, the 3-D GNSS observations are projected into LOS direction. The InSAR observations are averaged 3 by 3 near the station positions.

**NOTE:** the number of pixels used in calculating the averaged phase values at the GPS location depends on the resolution of input data.

Get daily position solutions for GNSS stations:

In [ ]:
#os.chdir(mint_dir)
displacement = {}
gnss_time_series = {}
gnss_time_series_std = {}
bad_stn = {}  #stations to toss
pixel_radius = 1   #number of InSAR pixels to average for comparison with GNSS

for counter,stn in enumerate(site_names):
    gnss_obj = gnss.get_gnss_class('UNR')(site = stn, data_dir = os.path.join(mintpy_dir,'GNSS'))
    gnss_obj.open()
        
    # count number of dates in time range
    gnss_obj.read_displacement()
    dates = gnss_obj.dates
    for i in range(insar_displacement.shape[0]):
        start_date = ifgs_date[i,0]
        end_date = ifgs_date[i,-1]
        
        range_days = (end_date - start_date).days
        gnss_count = np.histogram(dates, bins=[start_date,end_date])
        gnss_count = int(gnss_count[0])
        #print(gnss_count)

        # select GNSS stations based on data completeness, here we hope to select stations with data frequency of 1 day and no interruption
        if range_days == gnss_count-1:
        #if start_date in dates and end_date in dates:
            disp_gnss_time_series,disp_gnss_time_series_std,site_latlon = gnss_obj.get_los_displacement(os.path.join(mintpy_dir,'inputs/geometryGeo.h5'),
                                                                                                            start_date=start_date.strftime('%Y%m%d'),
                                                                                                            end_date=end_date.strftime('%Y%m%d'))[1:4]
            x_value = round((site_latlon[1] - W)/lon_step)
            y_value = round((site_latlon[0] - N)/lat_step)

            #displacement from insar observation in the gnss station, averaged
            #Caution: If you expand the radius parameter farther than the bounding grid it will break. 
            disp_insar = insar_displacement[i,
                                            y_value-pixel_radius:y_value+pixel_radius, 
                                            x_value-pixel_radius:x_value+pixel_radius]
            if np.isfinite(disp_insar).sum() == 0:
                break
            disp_insar = np.nanmean(disp_insar)

            disp_gnss_time_series = disp_gnss_time_series*1000 # convert unit from meter to mm
            disp_gnss_time_series_std = disp_gnss_time_series_std*1000
            gnss_time_series[(i,stn)] = disp_gnss_time_series
            gnss_time_series_std[(i,stn)] = disp_gnss_time_series_std
            displacement[(i,stn)] = list(site_latlon)
            disp_gnss = disp_gnss_time_series[-1] - disp_gnss_time_series[0]

            displacement[(i,stn)].append(disp_gnss)
            displacement[(i,stn)].append(disp_insar)
        else:
            try:
                bad_stn[i].append(stn)
            except:
                bad_stn[i] = [stn]
#os.chdir(cwd)

Do some data structure transformation:

In [ ]:
gnss_time_series = dict(sorted(gnss_time_series.items()))
gnss_time_series_std = dict(sorted(gnss_time_series_std.items()))
displacement = dict(sorted(displacement.items()))
bad_stn = dict(sorted(bad_stn.items()))

In [ ]:
gnss_time_series = pd.DataFrame.from_dict(gnss_time_series)
gnss_time_series_std = pd.DataFrame.from_dict(gnss_time_series_std)

In [ ]:
displacement = pd.DataFrame.from_dict(displacement,orient='index',
                                      columns=['lat','lon','gnss_disp','insar_disp'])
displacement.index = pd.MultiIndex.from_tuples(displacement.index,names=['ifg index','station'])

If there are less than 3 GNSS stations, don't conduct comparison:

In [ ]:
drop_index = []
for i in displacement.index.get_level_values(0).unique():
    if len(displacement.loc[i]) < 3:
        drop_index.append(i)
displacement=displacement.drop(drop_index)
# ifgs_date after drop for approach 1
ifgs_date_ap1=np.delete(ifgs_date,drop_index,axis=0)

All data needed for approach 1:

In [ ]:
displacement

**NOTE:** 
- A more general critterion is needed for GNSS station selection. Here the stations with uninterrupted data are selected while, in Secular Requirement Validation, stations are selected by data completeness and standard variation.

<a id='transient_gnss_insar'></a>
## Make GNSS and InSAR Relative Displacements

Here we randomly select one reference site and make both the GNSS and InSAR measurements relative to that reference to remove a constant offset.

In [ ]:
# reference GNSS stations to GNSS reference site
for i in displacement.index.get_level_values(0).unique():
    gnss_ref_site_name = random.choice(displacement.loc[i].index.unique())
    displacement.loc[i,'gnss_disp'] = displacement.loc[i,'gnss_disp'].values - displacement.loc[(i,gnss_ref_site_name),'gnss_disp']
    displacement.loc[i,'insar_disp'] = displacement.loc[i,'insar_disp'].values - displacement.loc[(i,gnss_ref_site_name),'insar_disp']
    ref_x_value = round((displacement.loc[(i,gnss_ref_site_name),'lon'] - W)/lon_step)
    ref_y_value = round((displacement.loc[(i,gnss_ref_site_name),'lat'] - N)/lat_step)

    ref_disp_insar = insar_displacement[i,
                                        ref_y_value-pixel_radius:ref_y_value+1+pixel_radius, 
                                        ref_x_value-pixel_radius:ref_x_value+1+pixel_radius]
    ref_disp_insar = np.nanmean(ref_disp_insar)
    insar_displacement[i] -= ref_disp_insar

Plot GNSS stations on InSAR displacement fields

In [ ]:
cmap = copy.copy(plt.get_cmap('RdBu'))
#cmap.set_bad(color='black')
vmin, vmax = np.nanmin(insar_displacement), np.nanmax(insar_displacement)
for i in displacement.index.get_level_values(0).unique():
    fig, ax = plt.subplots()
    img1 = ax.imshow(insar_displacement[i], cmap=cmap,vmin=vmin,vmax=vmax, interpolation='nearest', extent=(W, E, S, N))
    ax.set_title(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))
    cbar1 = fig.colorbar(img1, ax=ax)
    cbar1.set_label('LOS displacement [mm]')

    for stn in displacement.loc[i].index:
        lon,lat = displacement.loc[(i,stn),'lon'],displacement.loc[(i,stn),'lat']
        color = cmap((displacement.loc[(i,stn),'gnss_disp']-vmin)/(vmax-vmin))
        ax.scatter(lon,lat,s=8**2,color=color,edgecolors='k')
        ax.annotate(stn,(lon,lat),color='black')

<a id='transient_validation1'></a>
# NISAR Validation Approach 1: GNSS-InSAR Direct Comparison

<a id='transient_pair1'></a>
## Pair up GNSS stations and make measurement residuals

We first pair up all GNSS stations and compare the relative measurement from both GNSS and INSAR. 

In [ ]:
insar_disp = {}
gnss_disp = {}
ddiff_dist = {}
ddiff_disp = {}
abs_ddiff_disp = {}
for i in displacement.index.get_level_values(0).unique():
    displacement_i = displacement.loc[i]
    insar_disp_i = []
    gnss_disp_i = []
    ddiff_dist_i = []
    ddiff_disp_i = []

    for sta1 in displacement_i.index:
        for sta2 in displacement_i.index:
            if sta2 == sta1:
                break
            insar_disp_i.append(displacement_i.loc[sta1,'insar_disp']-displacement_i.loc[sta2,'insar_disp'])
            gnss_disp_i.append(displacement_i.loc[sta1,'gnss_disp']-displacement_i.loc[sta2,'gnss_disp'])
            ddiff_disp_i.append(gnss_disp_i[-1]-insar_disp_i[-1])
            g = pyproj.Geod(ellps="WGS84")
            _,_,distance = g.inv(displacement_i.loc[sta1,'lon'],displacement_i.loc[sta1,'lat'],
                                 displacement_i.loc[sta2,'lon'],displacement_i.loc[sta2,'lat'])
            distance = distance/1000 # convert unit from m to km
            ddiff_dist_i.append(distance)
    insar_disp[i]=np.array(insar_disp_i)
    gnss_disp[i]=np.array(gnss_disp_i)
    ddiff_dist[i]=np.array(ddiff_dist_i)
    ddiff_disp[i]=np.array(ddiff_disp_i)
    abs_ddiff_disp[i]=abs(np.array(ddiff_disp_i))

Plot to compare displacement from GNSS and InSAR:

In [ ]:
for i in displacement.index.get_level_values(0).unique():
    plt.figure(figsize=(11,7))
    disp_range = (min([*insar_disp[i],*gnss_disp[i]]),max([*insar_disp[i],*gnss_disp[i]]))
    plt.hist(insar_disp[i],bins=100,range=disp_range,color = "green",label='D_InSAR')
    plt.hist(gnss_disp[i],bins=100,range=disp_range,color="orange",label='D_GNSS', alpha=0.5)
    plt.legend(loc='upper right')
    plt.title(f"Displacements \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')} \n Number of station pairs used: {len(insar_disp[i])}")
    plt.xlabel('LOS Displacement (mm)')
    plt.ylabel('Number of Station Pairs')
    plt.show()

Plot Displacement Residuals Distribution:

In [ ]:
for i in displacement.index.get_level_values(0).unique():
    plt.figure(figsize=(11,7))
    plt.hist(ddiff_disp[i],bins = 100, color="darkblue",linewidth=1,label='D_gnss - D_InSAR')
    plt.legend(loc='upper right')
    plt.title(f"Residuals \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')} \n Number of stations pairs used: {len(ddiff_disp[i])}")
    plt.xlabel('Displacement Residual (mm)')
    plt.ylabel('N Stations')
    plt.show()

Plot Absolute Displacement Residuals As a Function of Distance:

In [ ]:
for i in displacement.index.get_level_values(0).unique():
    dist_th = np.linspace(min(ddiff_dist[i]),max(ddiff_dist[i]),100)
    acpt_error = 3*(1+np.sqrt(dist_th))
    plt.figure(figsize=(11,7))
    plt.scatter(ddiff_dist[i],abs_ddiff_disp[i],s=1)
    plt.plot(dist_th, acpt_error, 'r')
    plt.xlabel("Distance (km)")
    plt.ylabel("Amplitude of Displacement Residuals (mm)")
    plt.title(f"Residuals \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')} \n Number of stations pairs used: {len(ddiff_dist[i])}")
    plt.legend(["Measurement","Mission Reqiurement"])
    #plt.xlim(0,5)
    plt.show()

In [ ]:
ddiff_dist_ap1 = list(ddiff_dist.values())
abs_ddiff_disp_ap1 = list(abs_ddiff_disp.values())

We have got all needed data for approach 1:
- `ddiff_dist_ap1`: distance of GNSS pairs,
- `abs_ddiff_disp_ap1`: absolute value of measurement redisuals,
- `ifgs_date_ap1`: list of date pairs of two SAR images that form a interferogram.

<a id='transient_bin1'></a>
## Validate the requirement based on binned measurement residuals

In [ ]:
n_ifgs = len(ddiff_dist_ap1)

Bin all measurement residuals to check if they pass the requirements or not:

In [ ]:
n_bins = 10
bins = np.linspace(0.1,50.0,num=n_bins+1)

In [ ]:
n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
for i in range(n_ifgs):
    inds = np.digitize(ddiff_dist_ap1[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 3*(1+np.sqrt(ddiff_dist_ap1[i][inds==j]))# mission requirement for i-th ifgs and j-th bins
        rem = abs_ddiff_disp_ap1[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [ ]:
ratio = n_pass/n_all
thresthod = 0.683 
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod

<a id='transient_result1'></a>
## Result visualization

Convert the result to pandas DataFrame for better visulization:

In [ ]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

index = []
for i in range(len(ifgs_date_ap1)):
    index.append(ifgs_date_ap1[i,0].strftime('%Y%m%d')+'-'+ifgs_date_ap1[i,1].strftime('%Y%m%d'))

n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns,index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns,index=index)

Number of data points in each bin:

In [ ]:
n_all_pd

Number of data points that below the curve:

In [ ]:
n_pass_pd

Percentage of pass:

In [ ]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

<a id='transient_conclusion1'></a>
## Conclusion

In [ ]:
percentage = np.count_nonzero(ratio_pd['total']>thresthod)/n_ifgs
print(f"Percentage of interferograms passes the requirement: {percentage}")
if percentage >= 0.70:
    print('The interferogram stack passes the requirement.')
else:
    print('The interferogram stack fails the requirement.')

<div class="alert alert-warning">
Approach 1 final result for CentralValleyA144: around 79% of interferograms passes the requirement.
</div>

<a id='transient_validation2'></a>
# NISAR Validation Approach 2: Noise Level Validation

In this validation (Approach #2), we evaluate the estimated secular deformation rate (Requirements 658) or co-seismic displacement (Requirement 660) from time series processing or the individual unwrapped interferogram (Requirement 663) over selected cal/val areas with negligible deformation. Any estimated deformation should thus be treated as noise and our goal is to evaluate the significance of this noise. In general, noise in the modeled displacement or the unwrapped interferogram is anisotropic, but here we neglect this anisotropy. Also, we assume the noise is stationary.

We first randomly sample measurements and pair up sampled pixel measurements. For each pixel-pair, the difference of their measurement becomes:
$$d\left(r\right)=|(f\left(x\right)-f\left(x-r\right))|$$
Estimates of $d(r)$ from all pairs are binned according to the distance r. In each bin, $d(r)$ is assumed to be a normal distribution.

**Note:** Now we simply assume there is no deformation in this study area and time interval. But in fact, it is hard to find a enough large area without any deformation. An more realistic solution is to apply a mask to mask out deformed regions.

In [ ]:
n_ifgs = insar_displacement.shape[0]

Mask Pixels with Low Coherence (optional)

In [ ]:
#insar_displacement[insar_coherence <0.6] = np.nan

Plot the coherence and InSAR measurements:

In [ ]:
cmap = plt.get_cmap('gray')

for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.imshow(insar_coherence[i],cmap=cmap, interpolation='nearest',extent=(W, E, S, N))
    ax.set_title(f"Coherence \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    cbar1 = fig.colorbar(img1, ax=ax)
    cbar1.set_label('coherence')

In [ ]:
cmap = plt.get_cmap('RdBu')
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.imshow(insar_displacement[i], cmap=cmap, interpolation='nearest', extent=(W, E, S, N))
    ax.set_title(f"Interferogram \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    cbar1 = fig.colorbar(img1, ax=ax)
    cbar1.set_label('LOS displacement [mm]')

<a id='transient_pair2'></a>
## Randomly sample pixels and pair them up

Calculate the coordinate for every pixel:

In [ ]:
X0,Y0 = load_geo(atr)
X0_2d,Y0_2d = np.meshgrid(X0,Y0)

For each interferogram, randomly selected pixels need to be paired up. In order to keep measurements independent, different pixel pairs can not share same pixel. This is achieved by pairing up in sequence, i.e., pairing up pixel number 1 and number 2, 3 and 4...

In [ ]:
dist = []; rel_measure = []
for i in range(n_ifgs):
    dist_i, rel_measure_i = samp_pair(X0_2d,Y0_2d,insar_displacement[i],num_samples=1000000)
    dist.append(dist_i)
    rel_measure.append(rel_measure_i)

Show the statistical property of selected pixel pairs:

In [ ]:
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.hist(dist[i], bins=100)
    ax.set_title(f"Histogram of distance \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    ax.set_xlabel(r'Distance ($km$)')
    ax.set_ylabel('Frequency')
    ax.set_xlim(0,50)

In [ ]:
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    img1 = ax.hist(rel_measure[i], bins=100)
    ax.set_title(f"Histogram of Relative Measurement \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    ax.set_xlabel(r'Relative Measurement ($mm$)')
    ax.set_ylabel('Frequency')

In [ ]:
dist_th = np.linspace(0,50,100)
rqmt = 3*(1+np.sqrt(dist_th))
for i in range(n_ifgs):
    fig, ax = plt.subplots(figsize=[18, 5.5])
    ax.plot(dist_th, rqmt, 'r')
    ax.scatter(dist[i], rel_measure[i], s=1, alpha=0.25)
    ax.set_title(f"Comparation between Relative Measurement and Requirement Curve \n Date range {ifgs_date[i,0].strftime('%Y%m%d')}-{ifgs_date[i,1].strftime('%Y%m%d')}")
    ax.set_ylabel(r'Relative Measurement ($mm$)')
    ax.set_xlabel('Distance (km)')
    ax.set_xlim(0,50)

We have got data used of approach 2:
- `dist`: distance of pixel pairs,
- `rel_measure`: relative measurement of pixel pairs,
- `ifgs_date`: list of date pairs of two SAR images that form a interferogram.

<a id='transient_bin2'></a>
## Validate the requirement based on binned measurement residuals

In [ ]:
n_ifgs = len(dist)

Bin all measurement residuals to check if they pass the requirements or not:

In [ ]:
n_bins = 10
bins = np.linspace(0.1,50.0,num=n_bins+1)

In [ ]:
n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
#ratio = np.empty([n_ifgs,n_bins+1]) # ratio
# the final column is the ratio as a whole
for i in range(n_ifgs):
    inds = np.digitize(dist[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 3*(1+np.sqrt(dist[i][inds==j]))# mission requirement for i-th ifgs and j-th bins
        rem = rel_measure[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [ ]:
ratio = n_pass/n_all
mean_ratio = np.array([np.mean(ratio[:,:-1],axis=1)])
ratio = np.hstack((ratio,mean_ratio.T))
thresthod = 0.683
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod

<a id='transient_result2'></a>
## Result visualization

Convert the result to pandas DataFrame for better visulization:

In [ ]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

index = []
for i in range(len(ifgs_date)):
    index.append(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))

n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns+['mean'],index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns+['mean'],index=index)

Number of data points in each bin:

In [ ]:
n_all_pd

Number of data points that below the curve:

In [ ]:
n_pass_pd

Ratio of pass:

In [ ]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

<a id='transient_conclusion2'></a>
## Conclusion

Compared with percentage of total passed pairs, the mean value of percentage of passed pairs in all bin is a better indicator since it gives all bins same weight. 

In [ ]:
percentage = np.count_nonzero(ratio_pd['mean']>thresthod)/n_ifgs

In [ ]:
print(f"Percentage of interferograms passes the requirement (70%): {percentage}.")
if percentage >= 0.70:
    print('The interferogram stack passes the requirement.')
else:
    print('The interferogram stack fails the requirement.')

<div class="alert alert-warning">
Approach 2 final result for CentralValleyA144: 100% of interferograms passes the requirement.
</div>

<a id='transient_appendix'></a>
# Appendix: GNSS Position Plots

The relative position in LOS direction for all GNSS stations are plotted here.

In [ ]:
gnss_time_series[1]

In [ ]:
for i in range(len(gnss_time_series)):
    start_time_str = ifgs_date[i,0].strftime('%Y%m%d')
    end_time_str = ifgs_date[i,1].strftime('%Y%m%d')
    for stn in gnss_time_series[i].columns:
        print(f'Plotting GPS postion from {start_time_str} to {end_time_str} at station: {stn}')
        series = gnss_time_series[i,str(stn)]
        plt.figure(figsize=(15,5))
        plt.title(f"station name: {stn}")
        plt.scatter(pd.date_range(start=ifgs_date[i,0], end=ifgs_date[i,1]),series)
        plt.xlabel('Time')
        plt.ylabel('Relative position in LOS direction (mm)')
        plt.savefig(os.path.join(work_dir,start_time_str+'_'+end_time_str+'_'+stn+'.jpg'))
        plt.close()